In [1]:
import pandas as pd 
import numpy as np 
import pyodbc
import json
import datetime #处理日期型数据类型
from decimal import Decimal, getcontext #处理decimal的数据类型

# 第一步 生成嵌套文件

In [3]:
def GetData(tbl): #参数为数据表名
    try:
        # 1.创建mysql数据库连接对象connection
        conn = pyodbc.connect(
        r'DRIVER={ODBC Driver 17 for SQL Server};' 
        #注：driver这里这样写即表明是用driver连接而不是用dsn，如果这里直接写sqlserver会在linux连接不了，只能在windows连接，linux连接需要写上driver名字
        r'SERVER=localhost;'
        r'DATABASE=TPCH;'
        r'UID=sa;'
        r'PWD=*********'
        )

        cur = conn.cursor()
        sql = "SELECT * FROM " + tbl
        cur.execute(sql) #执行sql命令
        data = cur.fetchall() #获取数据
        cur.close() #关闭cursor
        conn.close() #关闭connection
        # # 循环读取元组数据，进行查看
        for row in data:
            print(row)
            print('\n')
            break
        return data
    except:
        print('MySQL connect fail...')

#利用Getdata函数，通过数据表名获取数据表中的数据
orders = GetData('ORDERS')
lineitem = GetData('LINEITEM')
    

(1, 370, 'O', Decimal('172799.49'), datetime.date(1996, 1, 2), '5-LOW          ', 'Clerk#000000951', 0, 'nstructions sleep furiously among ')


(1, 1552, 93, 1, Decimal('17.00'), Decimal('24710.35'), Decimal('0.04'), Decimal('0.02'), 'N', 'O', datetime.date(1996, 3, 13), datetime.date(1996, 2, 12), datetime.date(1996, 3, 22), 'DELIVER IN PERSON        ', 'TRUCK     ', 'egular courts above the')




In [4]:
#查看数据获取是否成功
print(len(orders))
print(len(orders[0]))
print(len(lineitem))
print(len(lineitem[0]))

15000
9
60175
16


In [5]:
#生成order和lineitem嵌套json
order_info = {}
data = json.loads(json.dumps(order_info))
for row in orders:
    tmp = {}
    namelist = ['O_ORDERKEY','O_CUSTKEY','O_ORDERSTATUS','O_TOTALPRICE','O_ORDERDATE',
                'O_ORDERPRIORITY','O_CLERK','O_SHIPPRIORITY','O_COMMENT']
    for i in range(len(row)):
        tmp[namelist[i]] =row[i]
        tmp['LINEITEM_INFO'] = {} #LINEITEM_INFO用于来放LINEITEM表中的信息
    data[row[0]] = tmp.copy()
for row in lineitem:
    tmp = {}
    namelist = ['L_ORDERKEY','L_PARTKEY','L_SUPPKEY','L_LINENUMBER','L_QUANTITY','L_EXTENDEDPRICE',
                'L_DISCOUNT','L_TAX','L_RETURNFLAG','L_LINESTATUS','L_SHIPDATE','L_COMMITDATE',
                'L_RECEIPTDATE','L_SHIPINSTRUCT','L_SHIPMODE','L_COMMENT']
    for i in range(len(row)):
        tmp[namelist[i]] =row[i]
    data.get(tmp.get('L_ORDERKEY')).get('LINEITEM_INFO')[row[3]] = tmp.copy() #将LINEITEM表中的信息放到刚刚的LINEITEM_INFO中去，用订单明细号L_LINENUMBER来做标题


#通过O_ORDERKEY，L_ORDERKEY 和L_LINENUMBER 进行检查
wrong = False
for key1 in data:
    if key1 != data[key1]['O_ORDERKEY']: #检查O_ORDERKEY和外部标题是否匹配
        print(key1+':O_ORDERKEY  erorr!!!!!')
        break
    for key2 in data[key1]['LINEITEM_INFO']:   #检查LINEITEM_INFO中的LINEITEM_INFO和其标题是否匹配
        if key2 != data[key1]['LINEITEM_INFO'][key2]['L_LINENUMBER']:
            print(key1 + ':L_LINENUMBER  erorr!!!!!')
            wrong = True
            break
        if data[key1]['O_ORDERKEY'] != data[key1]['LINEITEM_INFO'][key2]['L_ORDERKEY']: #检查O_ORDERKEY和L_ORDERKEY是否匹配
            print(key1 + ':Match  erorr!!!!!')
            wrong = True
            break
    if wrong:
        break




In [5]:
#注意还不可直接将字典data转为json，因为日期格式json格式不能自己处理
orders_info = json.dumps(data,sort_keys=True, ensure_ascii=False)

TypeError: Object of type date is not JSON serializable

In [ ]:
#日期型数据在转json格式时，会出现错误，于是重写json的类，当遇到日期格式时特殊处理，其余正常

class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.datetime):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(obj, datetime.date):
            return obj.strftime('%Y-%m-%d')
        else:
            return json.JSONEncoder.default(self, obj)

#在使用json.dumps时指定类即可解决日期格式，但会发现还有Decimal格式json类也会报错，需要进一步处理
orders_info = json.dumps(data,ensure_ascii=False,cls=ComplexEncoder)


TypeError: Object of type Decimal is not JSON serializable

In [6]:
#Decimal数据在json转换时也会出错，于是重写json的类，当遇到日期格式和decimal格式时特殊处理，其余正常
class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.datetime):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(obj, datetime.date):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, Decimal):
            return float(obj) #遇到decimal类型，使用float转换
        else:
            return json.JSONEncoder.default(self, obj)

#在使用json.dumps时指定类即可
orders_info = json.dumps(data,ensure_ascii=False,cls=ComplexEncoder)

In [ ]:
f = open(r'./orders_info.json', 'w+')
# 写数据
f.write(orders_info)
# 关闭文件
f.close()

# 第二步 分区

这里使用了两种方法进行分区：

1. 直接使用订单数来对订单号进行均分
2. 按总的商品数来对订单进行均分

## 按订单号均分

In [7]:
#按指定分区数量（P 代表 JSON 文件分区数量）将生成的 JSON 文件划分为指定数量（P）的分区，
#按订单数进行均分

p = 10 #假设分10组

with open('./orders_info.json', 'r') as f:
    orders_info = json.load(f)
len_tot = len(orders_info)

orders_div2 = {}
#按订单数平分
k = int(len_tot/p) #靠0取整，最后一组要记得取全部

keys = list(orders_info.keys())
for i in range(p):
    start = i*k
    if i == p-1:
        end = len(orders_info)
    else:
        end = (i+1)*k
    dic_tmp = {}
    for key1 in keys[start:end]:
        dic_tmp[key1] = orders_info[key1]
    name = 'group'+str(i+1)
    orders_div2[name] = dic_tmp

In [10]:
#检查分组是否正确
print(len(orders_div2))
print('\n')
keys = []
for key1 in orders_div2:
    # print('\n')
    print(key1)
    print(len(orders_div2[key1]))
    len_tmp = 0
    for key2 in orders_div2[key1]:
        len_tmp = len_tmp+len(orders_div2[key1][key2]["LINEITEM_INFO"])
    print('lineitemlen:{}'.format(len_tmp))
    keys.extend(orders_div2[key1].keys())
print('\n')
print(len(keys))
print(len(set(keys)))

10


group1
1500
lineitemlen:6005
group2
1500
lineitemlen:5952
group3
1500
lineitemlen:6016
group4
1500
lineitemlen:6173
group5
1500
lineitemlen:6055
group6
1500
lineitemlen:5986
group7
1500
lineitemlen:6078
group8
1500
lineitemlen:5949
group9
1500
lineitemlen:5964
group10
1500
lineitemlen:5997


15000
15000


## 按商品总数进行均分

In [11]:
## 按指定分区数量（P 代表 JSON 文件分区数量）将生成的 JSON 文件划分为指定数量（P）的分区
#按商品总数进行均分
p = 10 #均分10组

with open('./orders_info.json', 'r') as f:
    orders_info = json.load(f)
len_tot = 0
for key1 in orders_info:
    len_tot =len_tot + len(orders_info[key1]['LINEITEM_INFO']) #计算总的商品数

orders_div = {}
#按商品数平分去找订单数
k = int(len_tot/p) #靠0取整，最后一组要记得取全部
count0 = 0
for i in range(p-1):
    len_tmp = 0
    dic_tmp = {}
    count = 0
    for key1 in orders_info:
        count = count + 1 
        if count > count0:
            dic_tmp[key1] = orders_info[key1]
            len_tmp = len_tmp + len(orders_info[key1]['LINEITEM_INFO'])
            if len_tmp > k:
                count0 = count
                break
    name = 'group'+str(i+1)
    orders_div[name] = dic_tmp

#对于最后一组，全部都要包括
len_tmp = 0
dic_tmp = {}
count = 0
for key1 in orders_info:
    count = count + 1 
    if count > count0:
        dic_tmp[key1] = orders_info[key1]
name = 'group' +str(p)
orders_div[name] = dic_tmp

In [16]:
#检查分组是否正确
print(len(orders_div))
print('\n')
keys = []
for key1 in orders_div:
    # print('\n')
    print(key1)
    print(len(orders_div[key1]))
    len_tmp = 0
    for key2 in orders_div[key1]:
        len_tmp = len_tmp+len(orders_div[key1][key2]["LINEITEM_INFO"])
    print('lineitemlen:{}'.format(len_tmp))
    keys.extend(orders_div[key1].keys())
print('\n')
print(len(keys))
print(len(set(keys)))

10


group1
1503
lineitemlen:6018
group2
1516
lineitemlen:6023
group3
1504
lineitemlen:6018
group4
1458
lineitemlen:6018
group5
1493
lineitemlen:6018
group6
1511
lineitemlen:6023
group7
1485
lineitemlen:6020
group8
1515
lineitemlen:6020
group9
1515
lineitemlen:6020
group10
1500
lineitemlen:5997


15000
15000


# 第三步 探究数据是否对，用两种方法去计算总价格

In [13]:
with open('./orders_info.json', 'r') as f:
    orders_info = json.load(f)

#按"O_TOTALPRICE"去计算总价格
O_totprice = 0

for key1 in orders_info:
    O_totprice = O_totprice + orders_info[key1]['O_TOTALPRICE']


#按L_QUANTITY,L_EXTENDEDPRICE, L_DISCOUNT, L_TAX
L_totprice = 0
for key1 in orders_info:
    for key2 in orders_info[key1]['LINEITEM_INFO']:
        extendprice = orders_info[key1]['LINEITEM_INFO'][key2]["L_EXTENDEDPRICE"]
        discount = orders_info[key1]['LINEITEM_INFO'][key2]["L_DISCOUNT"]
        tax = orders_info[key1]['LINEITEM_INFO'][key2]["L_TAX"]
        L_totprice = L_totprice + extendprice*(1-discount)*(1+tax)
    

In [14]:
with open('./orders_info.json', 'r') as f:
    orders_info = json.load(f)

#按"O_TOTALPRICE"去计算总价格
O_totprice = 0
#按L_QUANTITY,L_EXTENDEDPRICE, L_DISCOUNT, L_TAX
L_totprice = 0
count = 0
for key1 in orders_info:
    tmp_Otot = orders_info[key1]['O_TOTALPRICE']
    O_totprice = O_totprice + tmp_Otot
    tmp_Ltot = 0
    for key2 in orders_info[key1]['LINEITEM_INFO']:
        # quantity = orders_info[key1]['LINEITEM_INFO'][key2]["L_QUANTITY"]
        extendprice = orders_info[key1]['LINEITEM_INFO'][key2]["L_EXTENDEDPRICE"]
        discount = orders_info[key1]['LINEITEM_INFO'][key2]["L_DISCOUNT"]
        tax = orders_info[key1]['LINEITEM_INFO'][key2]["L_TAX"]
        tmp_Ltot = tmp_Ltot + extendprice*(1-discount)*(1+tax)
    L_totprice = L_totprice + tmp_Ltot
    #可能由于税率或者打折的精度缺失导致差一些，因此做检查
    if abs(tmp_Otot-tmp_Ltot) >1: #用1做为阈值，低于1均不认为是计算错误 
        print('{}:error!!!!!'.format(key1))
        count = count+1
print(count)

0


In [15]:
print(O_totprice)
print(L_totprice)

2127396830.0200024
2127397347.0412745
